In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import os

In [19]:
data = pd.read_csv('data/clean/masterdata.csv')

In [20]:
data.head()

,"9th to 12th grade, no diploma",high school graduate (includes equivalency),"some college, no degree",associate's degree,bachelor's degree,graduate or professional degree,year,fips,geographic area name,median income (family),...,10 to 19 units,20 or more units,mobile home,"boat, rv, van, etc.",1-unit attached,boat rv van etc.,commute time,count,county,state
0,6.6,20.3,18.4,6.9,24.0,16.3,2010,6001,"Alameda County, California",85014,...,31836,89559,7154,612,NaN,NaN,12.456474,20,Alameda County,CA
1,9.8,30.5,28.5,9.5,13.2,5.8,2010,6005,"Amador County, California",65103,...,181,391,1341,2,NaN,NaN,9.893939,1,Amador County,CA
2,5.9,19.7,22.3,8.2,24.5,13.7,2010,6013,"Contra Costa County, California",91791,...,13795,32893,7131,231,NaN,NaN,13.799532,10,Contra Costa County,CA
3,10.8,23.2,22.6,7.6,13.4,6.3,2010,6019,"Fresno County, California",52306,...,8392,15799,14054,410,NaN,NaN,8.000660,2,Fresno County,CA
4,6.4,26.0,29.3,8.8,17.7,8.6,2010,6023,"Humboldt County, California",53221,...,1382,1672,5841,156,NaN,NaN,7.378587,2,Humboldt County,CA


In [21]:
data_2018 = data[data['year']==2018].drop(columns=['year', 'fips', 'geographic area name', 'county', 'state'])

In [22]:
data_2018.head()

,"9th to 12th grade, no diploma",high school graduate (includes equivalency),"some college, no degree",associate's degree,bachelor's degree,graduate or professional degree,median income (family),median income (nonfamily),total population,occupied housing units,...,3 or 4 units,5 to 9 units,10 to 19 units,20 or more units,mobile home,"boat, rv, van, etc.",1-unit attached,boat rv van etc.,commute time,count
3195,8608.0,25653.0,20474.0,6169.0,8060.0,6159.0,56402,23982,115098,45033,...,1626,1920,1171,819,7904,10,NaN,NaN,9.914334,95
3196,1245.0,3366.0,1807.0,893.0,510.0,434.0,49598,23952,13378,5236,...,192,122,0,28,1424,34,NaN,NaN,11.375766,4584
3197,3215.0,10924.0,8720.0,3316.0,3739.0,1951.0,56602,22679,49255,18670,...,1179,789,148,129,4200,3,NaN,NaN,8.486042,2
3198,3562.0,11546.0,4496.0,1844.0,2079.0,1208.0,46605,19215,37328,13428,...,864,872,86,87,2907,35,NaN,NaN,7.259966,4500
3199,2390.0,7666.0,3900.0,1732.0,1624.0,1055.0,48380,23085,31542,11210,...,235,501,133,351,2460,18,NaN,NaN,11.433327,216
